In [16]:
import numpy as np
import pandas as pd

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import tensorflow as tf
import pathlib

import hjs_ImageProcessing as ip

创建AUTOTUNE，计算机自动选取合适的线程数目

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

创建raw_dataset

In [3]:
data_root_orig = '/home/jameson/.keras/datasets/flower_photos'
data_root = pathlib.Path(data_root_orig)
print(data_root)

/home/jameson/.keras/datasets/flower_photos


In [4]:
my_dataset = ip.my_dataset(data_root)

In [5]:
my_dataset

<ZipDataset shapes: ((192, 192, 3), ()), types: (tf.float32, tf.int64)>

batch实例：

In [9]:
def batch_my_dataset(ds,batch_size,buffle_size):
   
    ds = ds.cache( )  #cache(filename='./cache.tf-data')
    ds =ds.shuffle(buffle_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    
    return ds

In [11]:
batch_dataset = batch_my_dataset(my_dataset,32,1000)

In [12]:
batch_dataset

<PrefetchDataset shapes: ((None, 192, 192, 3), (None,)), types: (tf.float32, tf.int64)>

data augumentation实例：

In [35]:
#copy from tensorflow
#data_augmentation.ipynb
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

In [13]:
#copy from tensorflow
#data_augmentation.ipynb
def convert(image, label):
  image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
  return image, label

def augment(image,label):
  image,label = convert(image, label)
  image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
  image = tf.image.resize_with_crop_or_pad(image, 250, 250) # Add  pixels of padding
  image = tf.image.random_crop(image, size=[192, 192, 3]) # Random crop back to 192x192
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

  return image,label

In [14]:
buffle_size = len(ip.get_image_paths(data_root))

augmented_train_batches = (
    my_dataset
    # Only train on a subset, so you can quickly see the effect.
    .take(100)
    .cache( )  #cache(filename='./cache.tf-data')
    .shuffle(buffle_size)
    .repeat()
    # The augmentation is added here.
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(32)
    .prefetch(AUTOTUNE)
) 

In [15]:
augmented_train_batches

<PrefetchDataset shapes: ((None, 192, 192, 3), (None,)), types: (tf.float32, tf.int64)>

拆分数据集:
方法一，直接对tensorflow.dataset类新操作

In [22]:
DATASET_SIZE = len(ip.get_image_paths(data_root))
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = my_dataset
full_dataset = full_dataset.shuffle(buffle_size)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

拆分数据集：方法二，借用sklearn

In [27]:
#算了吧